In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
TRAIN_PATH = "car_eval_train.csv"
TEST_PATH = "car_eval_test.csv"


df = pd.read_csv(TRAIN_PATH)
# print(df)

#ordered categories
ctg_order = {
    'buying': [['low', 'med', 'high', 'vhigh']],
    'maint': [['low', 'med', 'high', 'vhigh']],
    'doors': [['2', '3', '4', '5more']],
    'persons': [['2', '4', 'more']],
    'lug_boot': [['small', 'med', 'big']],
    'safety': [['low', 'med', 'high']],
    'class': [['unacc', 'acc', 'good', 'vgood']]
}

label_encoders = {}
for col in df.columns:
    oe = OrdinalEncoder(categories=ctg_order[col])
    df[col] = oe.fit_transform(df[[col]])
    label_encoders[col] = oe  # Store encoder for later use

X = df.drop(columns=["class"])
y = df["class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import balanced_accuracy_score

# random_state can be modified but submission file was generated based on model trained with random_state = 45
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

model = HistGradientBoostingClassifier()
model.fit(X_train, y_train)

predictions = model.predict(X_test)
# predictions
accuracy = balanced_accuracy_score(y_test, predictions)
print(f"Model Accuracy: {accuracy:.4f}")

Model Accuracy: 0.9962


In [9]:
model.fit(X, y)
#actual test data
test_data = pd.read_csv(TEST_PATH)
X_test = test_data.drop(columns=["id"])
for col in X_test.columns:
    X_test[col] = label_encoders[col].fit_transform(X_test[[col]])

test_data["output"] = model.predict(X_test)

test_data[["output"]] = (label_encoders["class"]
                        .inverse_transform(test_data[["output"]]))

output = test_data[["id", "output"]]
output.to_csv("car_eval_submission.csv", index=False)